importing modules

In [ ]:
import string
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

data gathering

In [ ]:
df = pd.read_csv('/content/archive(2).zip')

Data perprocessing and EDA

In [ ]:
df


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [ ]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [ ]:
df['text'].iloc[0]

"Subject: enron methanol ; meter # : 988291\r\nthis is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary\r\nflow data provided by daren } .\r\nplease override pop ' s daily volume { presently zero } to reflect daily\r\nactivity you can obtain from gas control .\r\nthis change is needed asap for economics purposes ."

In [ ]:
df['text'] = df['text'].apply(lambda x:x.replace('\r\n',' '))

In [ ]:
df['text'].iloc[0]

"Subject: enron methanol ; meter # : 988291 this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary flow data provided by daren } . please override pop ' s daily volume { presently zero } to reflect daily activity you can obtain from gas control . this change is needed asap for economics purposes ."

In [ ]:
stemmer = PorterStemmer()
corpus = []

stop_words = set(stopwords.words('english'))

for i in range(len(df)):
  text = df['text'].iloc[i].lower()
  text = text.translate(str.maketrans('','',string.punctuation)).split()
  text = [stemmer.stem(word) for word in text if word not in stop_words]
  text = ' '.join(text)
  corpus.append(text)

In [ ]:
df['text'].iloc[0]

"Subject: enron methanol ; meter # : 988291 this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary flow data provided by daren } . please override pop ' s daily volume { presently zero } to reflect daily activity you can obtain from gas control . this change is needed asap for economics purposes ."

In [ ]:
corpus[0]

'subject enron methanol meter 988291 follow note gave monday 4 3 00 preliminari flow data provid daren pleas overrid pop daili volum present zero reflect daili activ obtain ga control chang need asap econom purpos'

Split data

In [ ]:
vectorize = CountVectorizer()
X = vectorize.fit_transform(corpus).toarray()
Y = df.label_num

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

Model Selection

In [ ]:
clf = RandomForestClassifier()


0.9690821256038648

Model Training

In [ ]:
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

0.9690821256038648

Model Prediction

In [ ]:
to_predict_email = df.text.values[5170]
to_predict_email

'Subject: important online banking alert dear valued citizensr bank member , due to concerns , for the safety and integrity of the online banking community we have issued the following warning message . it has come to our attention that your citizensr bank account information needs to be updated as part of our continuing commitment to protect your account and to reduce the instance of fraud on our website . if you could please take 5 - 10 minutes out of your online experience and renew your records you will not run into any future problems with the online service . however , failure to confirm your records may result in your account suspension . once you have confirmed your account records your internet banking service will not be interrupted and will continue as normal . to confirm your bank account records please click here . note : this e - mail was sent on behalf of the online banking community , if you do not have an online banking account with charterr one then this message does 

In [ ]:
new_email = to_predict_email.lower().translate(str.maketrans('','',string.punctuation)).split()
new_email = [stemmer.stem(word) for word in new_email if word not in stop_words]
new_email = ' '.join(new_email)

email_corpus = [new_email]
x_email = vectorize.transform(email_corpus).toarray()

In [ ]:
clf.predict(x_email)

array([1])

In [ ]:
df.iloc[5170]

,5170
Unnamed: 0,4807
label,spam
text,Subject: important online banking alert dear v...
label_num,1


our model predict this email as spam